In [1]:
import cv2
from ultralytics import YOLO
import easyocr
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
# Initialize YOLO model and EasyOCR reader with exception handling
try:
    model = YOLO(r"D:/yolo/New folder/license_plate_detector.pt")
except Exception as e:
    print(f"Error loading YOLO model: {e}")
    exit(1)  # Exit if the model can't be loaded

video = cv2.VideoCapture(r"D:/yolo/car batch vedio 3/car 2/car 2-1.mp4")
if not video.isOpened():
    print("Error opening video file.")
    exit(1)  # Exit if video can't be opened

# Initialize EasyOCR
try:
    reader = easyocr.Reader(['en'])
except Exception as e:
    print(f"Error initializing EasyOCR: {e}")
    exit(1)  # Exit if the OCR reader can't be initialized

# Set the DPI for the Matplotlib figure
dpi = 100
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, frame = video.read()
    if not ret:
        print("End of video or unable to read the frame.")
        break  # Exit the loop if the video ends or can't read the frame

    # YOLO inference on the frame with error handling
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error during YOLO inference: {e}")
        continue  # Skip the frame if YOLO inference fails

    # Iterate over detected objects
    for result in results[0].boxes.data.tolist():
        try:
            x1, y1, x2, y2, score, class_id = result
            if score < 0.5:  # Confidence threshold to filter weak detections
                continue

            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Preprocess the car crop for OCR
            car_crop = frame[int(y1):int(y2), int(x1):int(x2)]
            gray_car = cv2.cvtColor(car_crop, cv2.COLOR_BGR2GRAY)
            gray_car = cv2.resize(gray_car, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
            _, thresh_car = cv2.threshold(gray_car, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # OCR on the cropped and preprocessed image
            license_plate_text = reader.readtext(thresh_car, detail=0)
            if license_plate_text:
                # Display detected text on the frame
                cv2.putText(frame, license_plate_text[0], (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        except Exception as e:
            print(f"Error during detection or OCR: {e}")
            continue  # Skip the current object if an error occurs

    # Convert the frame from BGR to RGB for Matplotlib
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Set the figure size based on the frame dimensions
    plt.figure(figsize=(frame_width / dpi, frame_height / dpi))
    plt.imshow(frame_rgb)
    plt.title('License Plate Detection and Recognition')
    plt.axis('off')
    plt.show()

    # Pause to simulate real-time video
    time.sleep(0.1)

# Release the video capture object and close all windows
video.release()
cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
import streamlit as st
import cv2
from ultralytics import YOLO
import easyocr
import numpy as np
import tempfile
import time

# Title and Description
st.title("License Plate Detection and Recognition")
st.write("This app detects and recognizes license plates in uploaded videos using YOLO and EasyOCR.")

# Video Upload
video_file = st.file_uploader("Upload a video", type=["mp4", "avi", "mov"])

# Detection settings
confidence_threshold = st.slider("Confidence Threshold", min_value=0.1, max_value=1.0, value=0.5)
pause_duration = st.slider("Pause Duration (seconds)", min_value=0.0, max_value=2.0, value=0.1)

if video_file:
    # Load YOLO model and EasyOCR
    model = YOLO("D:/yolo/New folder/license_plate_detector.pt")
    reader = easyocr.Reader(['en'])

    # Temporary file to store the video
    tfile = tempfile.NamedTemporaryFile(delete=False) 
    tfile.write(video_file.read()) 

    # Read uploaded video using OpenCV
    video = cv2.VideoCapture(tfile.name)

    # Run detection if the button is clicked
    if st.button("Run License Plate Detection"):
        while video.isOpened():
            ret, frame = video.read()
            if not ret:
                st.write("Video processing completed.")
                break

            # YOLO inference
            results = model(frame)

            for result in results[0].boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = result
                if score < confidence_threshold:
                    continue

                # Draw bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                # OCR on the cropped region
                car_crop = frame[int(y1):int(y2), int(x1):int(x2)]
                license_plate_text = reader.readtext(car_crop, detail=0)

                if license_plate_text:
                    cv2.putText(frame, license_plate_text[0], (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Convert BGR to RGB for displaying in Streamlit
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Show the processed frame
            st.image(frame_rgb, caption="Processed Frame", use_column_width=True)
            time.sleep(pause_duration)

    video.release()


In [ ]:
import streamlit as st
import cv2
from ultralytics import YOLO
import easyocr
import tempfile
import time
import numpy as np
import os

# Title and Description
st.title("License Plate Detection and Recognition")
st.write("This app detects and recognizes license plates in uploaded videos using YOLO and EasyOCR.")

# Video Upload
video_files = st.file_uploader("Upload video(s)", type=["mp4", "avi", "mov"], accept_multiple_files=True)

# Detection settings
confidence_threshold = st.slider("Confidence Threshold", min_value=0.1, max_value=1.0, value=0.5)
pause_duration = st.slider("Pause Duration (seconds)", min_value=0.0, max_value=2.0, value=0.1)

# Progress bar initialization
progress_bar = st.progress(0)

if video_files:
    # Load YOLO model and EasyOCR
    model = YOLO("D:/yolo/New folder/license_plate_detector.pt")
    reader = easyocr.Reader(['en'])

    for video_file in video_files:
        # Create a temporary file to store video
        tfile = tempfile.NamedTemporaryFile(delete=False)
        tfile.write(video_file.read())

        # Read uploaded video using OpenCV
        video = cv2.VideoCapture(tfile.name)

        # Get the total number of frames to calculate progress
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        processed_frames = 0
        detected_plates = []

        # Prepare to save the processed video
        frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        output_file = os.path.join(tempfile.gettempdir(), "processed_video.mp4")
        out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width, frame_height))

        # Run detection if the button is clicked
        st.write(f"Processing video: {video_file.name}")
        while video.isOpened():
            ret, frame = video.read()
            if not ret:
                break

            # YOLO inference
            results = model(frame)

            for result in results[0].boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = result
                if score < confidence_threshold:
                    continue

                # Draw bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                # OCR on the cropped region
                car_crop = frame[int(y1):int(y2), int(x1):int(x2)]
                license_plate_text = reader.readtext(car_crop, detail=0)

                if license_plate_text:
                    detected_plates.append(license_plate_text[0])
                    cv2.putText(frame, license_plate_text[0], (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                    # Convert BGR to RGB for displaying in Streamlit
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    st.image(frame_rgb, caption="Frame with Detected License Plate", use_column_width=True)

            # Save the processed frame into the video
            out.write(frame)

            # Update the progress bar
            processed_frames += 1
            progress_percentage = int((processed_frames / total_frames) * 100)
            progress_bar.progress(progress_percentage)

            time.sleep(pause_duration)

        # Close video streams
        video.release()
        out.release()

        # Show the detected license plates in a formatted way
        if detected_plates:
            st.markdown("### Detected License Plates:")
            for plate in detected_plates:
                st.markdown(f"- **{plate}**")

        # Provide download link for processed video
        st.markdown(f"### Download Processed Video:")
        st.video(output_file)

# Reset progress bar to 0 after completion
progress_bar.progress(0)
